In [1]:
import os
import torch
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"# Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0" # Set the GPU 2 to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [2]:
import os
import numpy as np
import random
import copy

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

from FeatureAcquisition import FeatureAcquisition
from Predictor import Predictor
from Generators import GaussianSampler

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from PartialVAE import PartialVAE
from metrics_dict import metrics_dict
from sklearn.metrics import auc as sklearn_auc


# Predefined functions

In [3]:
# 하나의 샘플에서 k개가 관측되었다고 가정
# k개의 관측을 임의로 배정

def sample_mask_uniform_K_per_sample(bs, d, min_K, max_K): # batch size, feature 개수, 최소 관측 샘플 수, 최대 관측 샘플 수
    m = np.zeros((bs, d), dtype=np.float32)
    Ks = np.random.randint(min_K, max_K+1, size=(bs,))
    for i, K in enumerate(Ks): # Ks의 index와 해당 index의 값
        idx = np.random.choice(d, size=K, replace=False)
        m[i, idx] = 1.0
    return m

In [4]:
def subsample_mask(mask):
    # Subsample by uniformly selecting removal probability, each feature
    # has that probability of being removed.
    # Multiply by true mask since it may be missing values to begin with.
    return (torch.rand_like(mask) > torch.rand_like(mask[:, :1])).float()*mask

In [5]:
def set_seed(numpy_seed, random_seed, torch_seed_cpu, torch_seed_cuda):
    random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    np.random.seed(numpy_seed)
    torch.manual_seed(torch_seed_cpu)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(torch_seed_cuda)
        torch.cuda.manual_seed_all(torch_seed_cuda)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [6]:
def train_generator(
    generator,
    train_loader,
    val_loader,
    epochs,
    optimizer_generator,
    save_path,
    obs_sigma=0.2,
    lr_factor=0.2,
    cooldown=0,
    min_lr=1e-7,
    scheduler_patience=5
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    generator.to(device)
    
    scheduler_generator = ReduceLROnPlateau(
        optimizer_generator,
        mode="min",
        factor=lr_factor,
        patience=scheduler_patience,
        cooldown=cooldown,
        min_lr=min_lr,
    )

    for epoch in range(epochs):
        # Train 
        generator.train()
        total_loss_generator, total_kl_generator, total_nll_generator = 0.0, 0.0, 0.0
        total_train_samples = 0.0

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            bs = xb.size(0)
            
            # generator 학습
            m_np = subsample_mask(xb)
            mb = torch.tensor(m_np, dtype=torch.float32, device=device)
            loss_generator, logs = generator.loss_func(
                xb,
                mb,
                obs_sigma=obs_sigma,
                n_samples=1,
            )
            
            optimizer_generator.zero_grad()
            loss_generator.backward()
            optimizer_generator.step()
            
            total_loss_generator += loss_generator.item() * bs
            total_kl_generator   += logs["KL"].item() * bs
            total_nll_generator  += logs["NLL_X"].item() * bs
            
            total_train_samples += bs
            
        train_loss_generator = total_loss_generator / total_train_samples
        train_kl   = total_kl_generator / total_train_samples
        train_nll  = total_nll_generator / total_train_samples

        # Validation 
        generator.eval()
        with torch.no_grad():
            total_metric_generator, total_kl_generator, total_nll_generator = 0.0, 0.0, 0.0
            total_val_samples = 0.0
            
            for xb, yb in val_loader:
                xb = xb.to(device)
                yb = yb.to(device)
                bs_val = xb.size(0)
                
                # generator 검증
                m_np = subsample_mask(xb)
                mb = torch.tensor(m_np, dtype=torch.float32, device=device)
                val_loss_generator, val_logs = generator.loss_func(
                    xb,
                    mb,
                    obs_sigma=obs_sigma,
                    n_samples=1,
                )
                total_metric_generator += val_loss_generator.item() * bs_val
                total_kl_generator += val_logs["KL"].item() * bs_val
                total_nll_generator += val_logs["NLL_X"].item() * bs_val
                
                total_val_samples += bs_val
            
            val_metric_generator = total_metric_generator / total_val_samples
            val_kl = total_kl_generator / total_val_samples
            val_nll = total_nll_generator / total_val_samples
            
            scheduler_generator.step(val_metric_generator)
            if val_metric_generator == scheduler_generator.best:
                best_state = copy.deepcopy(generator.state_dict())
                os.makedirs(save_path, exist_ok=True)
                torch.save(best_state, os.path.join(save_path, "best_model_generator.pt"))
    
        # 로그 출력
        current_lr_generator = optimizer_generator.param_groups[0]["lr"]
        print(
            f"Epoch {epoch+1}/{epochs} | "
            f"train_loss_generator={train_loss_generator:.4f} | "
            f"train_kl={train_kl:.4f} | "
            f"train_nll={train_nll:.4f} | "
            f"val_metric_generator={val_metric_generator:.4f} | "
            f"val_kl={val_kl:.4f} | "
            f"val_nll={val_nll:.4f} | "
            f"best={scheduler_generator.best:.4f} | "
            f"lr_generator={current_lr_generator:.6f}"
        )
        
    print(f"\nTraining complete. Best validation metric: {scheduler_generator.best:.4f}")



In [7]:
def run_feature_acquisition(
    predictor,
    generator,
    val_loader,
    metric_f,
    num_samples=10,
    alpha=1.0,
    gamma=0.5
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    predictor.eval()
    val_auc=0

    accs = []
    for xb, yb in val_loader:
        m_np = np.zeros_like(xb)
        xb=xb.to(device)
        yb=yb.to(device)
        
        mv = torch.tensor(m_np, dtype=torch.float32, device=device)
        val_metrics = [metric_f(predictor(xb, mv), yb)]


        for t in range(1, xb.shape[1]+1):
            FA = FeatureAcquisition(
                x=xb,
                m=mv,
                generative_model=generator,
                num_samples=num_samples,
                predictor=predictor,
                alpha=alpha,
                gamma=gamma
            )

            # mask 업데이트
            mv, _ = FA.acquire()
            val_metrics.append(metric_f(predictor(xb, mv), yb))

        val_metrics = np.array(val_metrics)
        val_auc += sklearn_auc(np.arange(xb.shape[1]+1), val_metrics)/(len(val_loader)*xb.shape[1])
        
    return val_auc

In [ ]:
def train_predictor(
    predictor,
    generator,
    train_loader,
    val_loader,
    epochs,
    optimizer_predictor,
    criterion,
    metric_f,
    save_path,
    lr_factor=0.2,
    cooldown=0,
    min_lr=1e-7,
    scheduler_patience=5
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    predictor.to(device)
    generator.to(device)

    scheduler_predictor = ReduceLROnPlateau(
        optimizer_predictor,
        mode="max",
        factor=lr_factor,
        patience=scheduler_patience,
        cooldown=cooldown,
        min_lr=min_lr,
    )
    
    for epoch in range(epochs):
        # Train 
        predictor.train()
        total_loss_predictor = 0.0
        total_train_samples = 0.0
        

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            bs = xb.size(0)

            # predictor 학습
            m_np = subsample_mask(xb)
            mb = torch.tensor(m_np, dtype=torch.float32, device=device)

            logits = predictor(xb, mb)
            loss_predictor = criterion(logits, yb)

            optimizer_predictor.zero_grad()
            loss_predictor.backward()
            optimizer_predictor.step()

            total_loss_predictor += loss_predictor.item() * bs
            
            total_train_samples += bs
          
            
        train_loss_predictor = total_loss_predictor / total_train_samples

        # Validation 
        predictor.eval()
        generator.eval()
        with torch.no_grad():
            total_metric_predictor = 0.0
            total_val_samples = 0.0
            
            for xb, yb in val_loader:
                xb=xb.to(device)
                yb=yb.to(device)
                bs_val = xb.size(0)
                
                # predictor 검증
                m_np = subsample_mask(xb)
                mb = torch.tensor(m_np, dtype=torch.float32, device=device)
                total_metric_predictor += metric_f(predictor(xb, mb), yb) * bs_val
                
                
                total_val_samples += bs_val
            
            val_metric_predictor = total_metric_predictor / total_val_samples
            val_auc = run_feature_acquisition(predictor, generator, val_loader, metric_f)

            scheduler_predictor.step(val_auc)
            if val_auc == scheduler_predictor.best:
                best_state = copy.deepcopy(predictor.state_dict())
                os.makedirs(save_path, exist_ok=True)
                torch.save(best_state, os.path.join(save_path,"best_model_predictor.pt"))

    
        # 로그 출력
        current_lr_predictor = optimizer_predictor.param_groups[0]["lr"]
        print(
            f"Epoch {epoch+1}/{epochs} | "
            f"train_loss_predictor={train_loss_predictor:.4f} | "
            f"Val Accuracy: {val_auc:.4f}|{scheduler_predictor.best:.4f}, Val Metric_predictor: {val_metric_predictor:.4f} | "
            f"lr_predictor={current_lr_predictor:.6f}"
        )
        
    val_auc = run_feature_acquisition(predictor, generator, val_loader, metric_f)
    os.makedirs(save_path, exist_ok=True)
    torch.save(val_auc, os.path.join(save_path, "val_auc.pt"))
    print(f"\nTraining complete, Zero Acquisition AUC: {val_auc:.3f}")


In [9]:
# def train_generator(
#     generator,
#     train_loader,
#     X_val,
#     D, # feature 개수
#     epochs,
#     optimizer,
#     obs_sigma=0.2,
#     lr_factor=0.2,
#     cooldown=0,
#     min_lr=1e-7,
#     scheduler_patience=5
# ):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     generator.to(device)
#     X_val = X_val.to(device)

#     scheduler = ReduceLROnPlateau(
#         optimizer,
#         mode="min", # loss 기준이기 때문 
#         factor=lr_factor,
#         patience=scheduler_patience,
#         cooldown=cooldown,
#         min_lr=min_lr,
#     )

#     best_val_loss = float("inf")
#     best_state = None

#     for ep in range(epochs):
#         # Train
#         generator.train()
#         total_loss, total_kl, total_nll = 0.0, 0.0, 0.0
#         count = 0

#         for xb, _ in train_loader:
#             xb = xb.to(device).float()
#             bs = xb.size(0)

#             m_np = sample_mask_uniform_K_per_sample(
#                 bs=bs,
#                 d=D,
#                 min_K=1,
#                 max_K=D,
#             )
#             mb = torch.tensor(m_np, dtype=torch.float32, device=device)

#             loss, logs = generator.loss_func(
#                 xb,
#                 mb,
#                 obs_sigma=obs_sigma,
#                 n_samples=1,
#             )

#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             total_loss += loss.item() * bs
#             total_kl   += logs["KL"].item() * bs
#             total_nll  += logs["NLL_X"].item() * bs
#             count      += bs

#         train_loss = total_loss / count
#         train_kl   = total_kl / count
#         train_nll  = total_nll / count

#         # Validation
#         generator.eval()
#         with torch.no_grad():
#             bs_val = X_val.size(0)
#             m_np = sample_mask_uniform_K_per_sample(
#                 bs=bs_val,
#                 d=D,
#                 min_K=1,
#                 max_K=D,
#             )
#             mv = torch.tensor(m_np, dtype=torch.float32, device=device)

#             val_loss_tensor, val_logs = generator.loss_func(
#                 X_val,
#                 mv,
#                 obs_sigma=obs_sigma,
#                 n_samples=1,
#             )

#             val_loss = val_loss_tensor.item()
#             val_kl   = val_logs["KL"].item()
#             val_nll  = val_logs["NLL_X"].item()

#         # 스케줄러 업데이트 (val_loss 기준)
#         scheduler.step(val_loss)
#         current_lr = optimizer.param_groups[0]["lr"]

#         print(
#             f"[PVAE ep {ep:02d}] "
#             f"train_loss={train_loss:.4f}  train_KL={train_kl:.4f}  train_NLL_X={train_nll:.4f} | "
#             f"val_loss={val_loss:.4f}  val_KL={val_kl:.4f}  val_NLL_X={val_nll:.4f} | "
#             f"lr={current_lr:.6f}"
#         )

#         # best model 저장 (val_loss 최소 기준)
#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             best_state = copy.deepcopy(generator.state_dict())

#     # 가장 좋은 val_loss 기준으로 weight 복원
#     if best_state is not None:
#         generator.load_state_dict(best_state)

#     print(f"Best val_loss = {best_val_loss:.4f}")

# CUBE

In [10]:
ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, "data", "cube")

X_train = torch.load(f"{DATA_DIR}/X_train_cdf.pt").float()
y_train = torch.load(f"{DATA_DIR}/y_train.pt").long()

X_val   = torch.load(f"{DATA_DIR}/X_val_cdf.pt").float()
y_val   = torch.load(f"{DATA_DIR}/y_val.pt").long()

X_test = torch.load(f"{DATA_DIR}/X_test_cdf.pt").float()
y_test = torch.load(f"{DATA_DIR}/y_test.pt").long()

dataset_dict = torch.load(f"{DATA_DIR}/dataset_dict.pt")

In [11]:
batch_size = 256
epochs = 100
lr = 1e-3
weight_decay = 1e-4

dataset_name = dataset_dict["dataset"]
num_con_features = dataset_dict["num_con_features"]
num_cat_features = dataset_dict["num_cat_features"]
num_classes = dataset_dict["out_dim"]
save_path = os.path.join(ROOT_DIR, "saved_models", dataset_name)


pvae = PartialVAE(
    input_type="continuous",
    num_con_features=num_con_features,
    num_cat_features=num_cat_features,
    hidden_dim_con=30,
    most_categories=max(1, 0),   # 내부 차원 계산을 위해 최소 1
    c_dim=16,
    hid_enc=100,
    hid_dec=100,
    latent_dim=30
)
optimizer_pvae = AdamW(pvae.parameters(), lr=lr, weight_decay=weight_decay)

train_ds = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
val_ds = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_ds, batch_size=len(X_val), shuffle=False, drop_last=False)

train_generator(
    generator=pvae,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=epochs,
    optimizer_generator=optimizer_pvae,
    save_path=save_path,
    obs_sigma=0.2,
    lr_factor=0.2,
    cooldown=0,
    min_lr=1e-7,
    scheduler_patience=5
)

/tmp/ipykernel_413029/716532247.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)
/tmp/ipykernel_413029/716532247.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)


Epoch 1/100 | train_loss_generator=-3938073.5672 | train_kl=68.3702 | train_nll=-3938141.9322 | val_metric_generator=-1769292544.0000 | val_kl=126.9141 | val_nll=-1769292672.0000 | best=-1769292544.0000 | lr_generator=0.001000
Epoch 2/100 | train_loss_generator=-49380696.2288 | train_kl=138.1320 | train_nll=-49380834.5278 | val_metric_generator=-99087552.0000 | val_kl=146.7639 | val_nll=-99087696.0000 | best=-1769292544.0000 | lr_generator=0.001000
Epoch 3/100 | train_loss_generator=-231398163.2387 | train_kl=145.6222 | train_nll=-231398308.2891 | val_metric_generator=-86634384.0000 | val_kl=144.1384 | val_nll=-86634528.0000 | best=-1769292544.0000 | lr_generator=0.001000
Epoch 4/100 | train_loss_generator=-711113918.7348 | train_kl=148.9742 | train_nll=-711114059.7852 | val_metric_generator=-81843077120.0000 | val_kl=140.9916 | val_nll=-81843077120.0000 | best=-81843077120.0000 | lr_generator=0.001000
Epoch 5/100 | train_loss_generator=-1618445853.0901 | train_kl=145.9473 | train_nll=

## Partial VAE

In [12]:
batch_size = 256
epochs = 100
lr = 1e-3
weight_decay = 1e-4

dataset_name = dataset_dict["dataset"]
feature_dim = dataset_dict["num_con_features"] + dataset_dict["num_cat_features"]
num_classes = dataset_dict["out_dim"]
metric_f = metrics_dict[dataset_dict["metric"]]
save_path = os.path.join(ROOT_DIR, "saved_models", dataset_name)


predictor = Predictor(feature_dim=feature_dim, num_classes=num_classes, hidden_dim=128)
optimizer_predictor = AdamW(predictor.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

pvae = PartialVAE(
    input_type="continuous",
    num_con_features=20,
    num_cat_features=0,
    hidden_dim_con=30,
    most_categories=max(1, 0),   # 내부 차원 계산을 위해 최소 1
    c_dim=16,
    hid_enc=100,
    hid_dec=100,
    latent_dim=30
)
# 훈련한 generator 불러오기
pvae.load_state_dict(torch.load(os.path.join(save_path, "best_model_generator.pt")))

train_ds = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
val_ds = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_ds, batch_size=len(X_val), shuffle=False, drop_last=False)

train_predictor(
    predictor=predictor,
    generator=pvae,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=epochs,
    optimizer_predictor=optimizer_predictor,
    criterion=criterion,
    metric_f=metric_f,
    save_path=save_path,
    lr_factor=0.2,
    cooldown=0,
    min_lr=1e-7,
    scheduler_patience=5
)

/tmp/ipykernel_413029/773552123.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)
/tmp/ipykernel_413029/773552123.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)


Epoch 1/100 | train_loss_predictor=1.5031 | Val Accuracy: 0.1450|0.1450, Val Metric_predictor: 0.5802lr_predictor=0.001000


/tmp/ipykernel_413029/773552123.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)
/tmp/ipykernel_413029/773552123.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)


Epoch 2/100 | train_loss_predictor=1.1634 | Val Accuracy: 0.1295|0.1450, Val Metric_predictor: 0.6015lr_predictor=0.001000


/tmp/ipykernel_413029/773552123.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)
/tmp/ipykernel_413029/773552123.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)


...

Epoch 98/100 | train_loss_predictor=1.0359 | Val Accuracy: 0.1241|0.1450, Val Metric_predictor: 0.6094lr_predictor=0.000000


/tmp/ipykernel_413029/773552123.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)
/tmp/ipykernel_413029/773552123.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)


Epoch 99/100 | train_loss_predictor=1.0414 | Val Accuracy: 0.1234|0.1450, Val Metric_predictor: 0.6043lr_predictor=0.000000


/tmp/ipykernel_413029/773552123.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)
/tmp/ipykernel_413029/773552123.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mb = torch.tensor(m_np, dtype=torch.float32, device=device)


Epoch 100/100 | train_loss_predictor=1.0359 | Val Accuracy: 0.1229|0.1450, Val Metric_predictor: 0.6041lr_predictor=0.000000

Training complete, Zero Acquisition AUC: 0.125


In [ ]:
predictor.eval()
N = X_test.size(0)
D = X_test.size(1)
x_np = X_test.cpu().numpy()
m_np = np.zeros((N, D), dtype=np.float32)

result = []

accs = run_feature_acquisition(
    predictor=predictor,
    generator=pvae,
    X_test=X_test,
    y_test=y_test,
    x_np=x_np,
    m_np=m_np,
    D=D,
    num_samples=10,
    alpha=1,
    gamma=0.5
)

result.append(accs)

Step 1/20 | Acc: 0.3142
Step 2/20 | Acc: 0.5485
Step 3/20 | Acc: 0.7354
Step 4/20 | Acc: 0.8492
Step 5/20 | Acc: 0.9076
Step 6/20 | Acc: 0.9382
Step 7/20 | Acc: 0.9542
Step 8/20 | Acc: 0.9593
Step 9/20 | Acc: 0.9614
Step 10/20 | Acc: 0.9619
Step 11/20 | Acc: 0.9621
Step 12/20 | Acc: 0.9620
Step 13/20 | Acc: 0.9621
Step 14/20 | Acc: 0.9618
Step 15/20 | Acc: 0.9618
Step 16/20 | Acc: 0.9615
Step 17/20 | Acc: 0.9612
Step 18/20 | Acc: 0.9608
Step 19/20 | Acc: 0.9601
Step 20/20 | Acc: 0.9597


In [ ]:
result = np.array(result, dtype=np.float32)

num_runs, D = result.shape
print(result.shape)
row_means = result.mean(axis=1)
overall_mean = row_means.mean()

std = row_means.std()

col_means = result.mean(axis=0)

print("전체 평균:", overall_mean)
print("표준편차:", std)
print("acquisition 별 평균):", col_means)

# 시각화
xs = np.arange(1, D + 1)

plt.figure(figsize=(5, 4))
plt.plot(xs, col_means, marker='o')
plt.xlabel('Acquisition No.', fontsize=11)
plt.ylabel('ACCURACY', fontsize=11)
plt.title('Cube', fontsize=12)
plt.xticks(np.arange(0, D+1, 2))
plt.xlim(0, D + 1)
plt.ylim(0.4, 1.0)
plt.grid(True, alpha=0.3)

# 확대 영역
ax = plt.gca()
axins = inset_axes(ax, width="50%", height="50%", loc='center right')

axins.plot(xs, col_means, marker='o')
axins.set_xlim(6, 8)
axins.set_ylim(0.92, 0.98)
axins.grid(True, alpha=0.3)
axins.tick_params(labelsize=8)

plt.show()

mean_acc = np.nanmean(col_means)
print(f"Mean ACC over all acquisitions = {mean_acc:.4f}")

(1, 20)
전체 평균: 0.88714993
표준편차: 0.0
acquisition 별 평균): [0.3142 0.5485 0.7354 0.8492 0.9076 0.9382 0.9542 0.9593 0.9614 0.9619
 0.9621 0.962  0.9621 0.9618 0.9618 0.9615 0.9612 0.9608 0.9601 0.9597]


AttributeError: 'NoneType' object has no attribute '_get_renderer'

<Figure size 500x400 with 2 Axes>

Mean ACC over all acquisitions = 0.8871


## ACFlow

# Bank Marketing

## Partial VAE

## ACFlow

# California Housing

In [ ]:
ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, "data", "california_housing")

X_train = torch.load(f"{DATA_DIR}/X_train_cdf.pt").float()
y_train = torch.load(f"{DATA_DIR}/y_train.pt").long()

X_val   = torch.load(f"{DATA_DIR}/X_val_cdf.pt").float()
y_val   = torch.load(f"{DATA_DIR}/y_val.pt").long()

X_test = torch.load(f"{DATA_DIR}/X_test_cdf.pt").float()
y_test = torch.load(f"{DATA_DIR}/y_test.pt").long()

In [ ]:
batch_size = 256
epochs = 100
lr = 1e-3
weight_decay = 1e-4
D = 8  # feature 개수 고정

predictor = Predictor(feature_dim=8, num_classes=4, hidden_dim=128)
optimizer = AdamW(predictor.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

train_ds = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)

train_predictor(
    predictor=predictor,
    train_loader=train_loader,
    X_val=X_val,
    y_val=y_val,
    D=D,
    epochs=epochs,
    optimizer=optimizer,
    criterion=criterion
)

Epoch 1/100 | train_loss=1.3093 | val_loss=1.2060 | val_acc=0.4370 | lr=0.001000
Epoch 2/100 | train_loss=1.2101 | val_loss=1.1537 | val_acc=0.4714 | lr=0.001000
Epoch 3/100 | train_loss=1.1858 | val_loss=1.1568 | val_acc=0.4680 | lr=0.001000
Epoch 4/100 | train_loss=1.1785 | val_loss=1.1428 | val_acc=0.4612 | lr=0.001000
Epoch 5/100 | train_loss=1.1597 | val_loss=1.1220 | val_acc=0.4767 | lr=0.001000
Epoch 6/100 | train_loss=1.1466 | val_loss=1.1217 | val_acc=0.5029 | lr=0.001000
Epoch 7/100 | train_loss=1.1418 | val_loss=1.1207 | val_acc=0.4913 | lr=0.001000
Epoch 8/100 | train_loss=1.1353 | val_loss=1.1169 | val_acc=0.5015 | lr=0.001000
Epoch 9/100 | train_loss=1.1353 | val_loss=1.1090 | val_acc=0.4864 | lr=0.001000
Epoch 10/100 | train_loss=1.1294 | val_loss=1.1201 | val_acc=0.4748 | lr=0.001000
Epoch 11/100 | train_loss=1.1344 | val_loss=1.1062 | val_acc=0.4927 | lr=0.001000
Epoch 12/100 | train_loss=1.1199 | val_loss=1.1038 | val_acc=0.5000 | lr=0.000200
Epoch 13/100 | train_loss

In [ ]:
batch_size = 256
epochs = 100
lr = 1e-3
weight_decay = 1e-4

pvae = PartialVAE(
    input_type="continuous",
    num_con_features=8,
    num_cat_features=0,
    hidden_dim_con=30,
    most_categories=max(1, 0),   # 내부 차원 계산을 위해 최소 1
    c_dim=16,
    hid_enc=100,
    hid_dec=100,
    latent_dim=30
)
optimizer = AdamW(pvae.parameters(), lr=lr, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

train_generator(
    generator=pvae,
    train_loader=train_loader,
    X_val=X_val,
    D=D, # feature 개수
    epochs=epochs,
    optimizer=optimizer,
    obs_sigma=0.2,
    lr_factor=0.2,
    cooldown=0,
    min_lr=1e-7,
    scheduler_patience=5
)

[PVAE ep 00] train_loss=11.7871  train_KL=1.8051  train_NLL_X=9.9820 | val_loss=9.3578  val_KL=1.9682  val_NLL_X=7.3896 | lr=0.001000
[PVAE ep 01] train_loss=8.4565  train_KL=2.1349  train_NLL_X=6.3216 | val_loss=8.0482  val_KL=2.3415  val_NLL_X=5.7067 | lr=0.001000
[PVAE ep 02] train_loss=7.6641  train_KL=2.3469  train_NLL_X=5.3172 | val_loss=7.6073  val_KL=2.5025  val_NLL_X=5.1048 | lr=0.001000
[PVAE ep 03] train_loss=7.2944  train_KL=2.4704  train_NLL_X=4.8240 | val_loss=7.2036  val_KL=2.6113  val_NLL_X=4.5923 | lr=0.001000
[PVAE ep 04] train_loss=7.0381  train_KL=2.6366  train_NLL_X=4.4015 | val_loss=7.0186  val_KL=2.8248  val_NLL_X=4.1938 | lr=0.001000
[PVAE ep 05] train_loss=6.7740  train_KL=2.6870  train_NLL_X=4.0870 | val_loss=6.7994  val_KL=2.7823  val_NLL_X=4.0171 | lr=0.001000
[PVAE ep 06] train_loss=6.6118  train_KL=2.7530  train_NLL_X=3.8587 | val_loss=6.4991  val_KL=2.7722  val_NLL_X=3.7270 | lr=0.001000
[PVAE ep 07] train_loss=6.5102  train_KL=2.7808  train_NLL_X=3.7293 

In [ ]:
predictor.eval()
N = X_test.size(0)
D = X_test.size(1)
x_np = X_test.cpu().numpy()
m_np = np.zeros((N, D), dtype=np.float32)

result = []

accs = run_feature_acquisition(
    predictor=predictor,
    generator=pvae,
    X_test=X_test,
    y_test=y_test,
    x_np=x_np,
    m_np=m_np,
    D=D,
    num_samples=10,
    alpha=1,
    gamma=0.5
)

result.append(accs)

Step 1/8 | Acc: 0.4782
Step 2/8 | Acc: 0.5068
Step 3/8 | Acc: 0.5557
Step 4/8 | Acc: 0.5577
Step 5/8 | Acc: 0.5799
Step 6/8 | Acc: 0.5950
Step 7/8 | Acc: 0.6114
Step 8/8 | Acc: 0.6361


In [ ]:
result = np.array(result, dtype=np.float32)

num_runs, D = result.shape
row_means = result.mean(axis=1)
overall_mean = row_means.mean()

std = row_means.std()

col_means = result.mean(axis=0)

print("전체 평균:", overall_mean)
print("표준편차:", std)
print("acquisition 별 평균):", col_means)

# 시각화
xs = np.arange(1, D + 1)

plt.figure(figsize=(5, 4))
plt.plot(xs, col_means, marker='o')
plt.xlabel('Acquisition No.', fontsize=11)
plt.ylabel('ACCURACY', fontsize=11)
plt.title('California Housing', fontsize=12)
plt.xticks(np.arange(0, D+1, 2))
plt.xlim(0, D + 1)
plt.ylim(0.4, 1.0)
plt.grid(True, alpha=0.3)

# 확대 영역
ax = plt.gca()
axins = inset_axes(ax, width="30%", height="30%", loc='center right')

axins.plot(xs, col_means, marker='o')
axins.set_xlim(6, 8)
axins.set_ylim(0.92, 0.98)
axins.grid(True, alpha=0.3)
axins.tick_params(labelsize=8)

plt.show()

mean_acc = np.nanmean(col_means)
print(f"Mean ACC over all acquisitions = {mean_acc:.4f}")

전체 평균: 0.5651041
표준편차: 0.0
acquisition 별 평균): [0.47819766 0.50678295 0.55571705 0.55765504 0.57994187 0.5949612
 0.6114341  0.6361434 ]
Mean ACC over all acquisitions = 0.5651


## Partial VAE

## ACFlow

# MiniBooNE

## Partial VAE

## ACFlow

# MNIST

## Partial VAE

## ACFlow

# Fashion MNIST

## Partial VAE

## ACFlow

# METABRIC

## Partial VAE

## ACFlow

# TCGA

## Partial VAE

## ACFlow